In [1]:
#import library
from pandas_gbq import read_gbq
import pandas as pd
import numpy as np
import os
import datetime
import ssl
import logging

/Users/sye/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
from datetime import date, timedelta

# Get today's date
today = date.today()
idx = (today.weekday() + 1) % 7

# Current week (CY) range
end_date = today - timedelta(days=(7 + idx - 6))
start_date = end_date - timedelta(days=6)

# Prior period (PP) range
pp_end_date = start_date - timedelta(days=1)
pp_start_date = pp_end_date - timedelta(days=13)

# Corresponding week last year (CWLY and PWLY)
cwly_date = pp_end_date - timedelta(days=364) + timedelta(days=7)
pwly_date = pp_end_date - timedelta(days=364)

# Output all dates
(end_date, start_date, pp_end_date, pp_start_date, cwly_date, pwly_date)


(datetime.date(2025, 7, 26),
 datetime.date(2025, 7, 20),
 datetime.date(2025, 7, 19),
 datetime.date(2025, 7, 6),
 datetime.date(2024, 7, 27),
 datetime.date(2024, 7, 20))

In [3]:
from custom_query import read_sql_query, sql_files

# Read SQL queries from files
queries = {key: read_sql_query(path) for key, path in sql_files.items()}

# Execute queries if all were successfully reada
if all(queries.values()):
    try:
        df_weekly_note = read_gbq(queries["weekly_note"], project_id='pcln-pl-airanalytics-prod')
        finance_data = read_gbq(queries["finance_data"], project_id='pcln-pl-airanalytics-prod')
        deal_share = read_gbq(queries["deal_share"], project_id='pcln-pl-airanalytics-prod')
        tsa_data = read_gbq(queries["tsa_data"], project_id='pcln-pl-airanalytics-prod')
        direct_parity_data = read_gbq(queries["direct_parity_data"], project_id='pcln-pl-airanalytics-prod')
        meta_parity_data = read_gbq(queries["meta_parity_data_query"], project_id='pcln-pl-airanalytics-prod')
        bookability_data = read_gbq(queries["bookability_query"], project_id='pcln-pl-airanalytics-prod')
        roi_data = read_gbq(queries["roi_query"], project_id='pcln-pl-airanalytics-prod')
        # dau_data = read_gbq(queries["dau_query"], project_id='pcln-pl-airanalytics-prod')
        midt_data = read_gbq(queries["midt_query"], project_id='pcln-pl-airanalytics-prod')
        sem_data = read_gbq(queries["sem_query"], project_id='pcln-pl-airanalytics-prod')
        print("SQL queries executed successfully.")
    except Exception as e:
        print(f"Failed to execute SQL queries: {e}")



Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
Downloading: 100%|██████████|
SQL queries executed successfully.


## make a copy of dataset

In [4]:
# make a copy of every data set
df_finance=finance_data.copy()
df_direct_parity=direct_parity_data.copy()
df_meta_parity = meta_parity_data.copy()
roi=roi_data.copy()
# dau=dau_data.copy()
df_deal_share=deal_share.copy()
df_bookability=bookability_data.copy()
df_weekly=df_weekly_note.copy()
df_weekly.columns = df_weekly.columns.str.lower()
# df_midt=midt_data.copy()
# df_sem=sem_data.copy()


In [5]:

base_dir = '../../data_file/'

# Check if directory exists
if not os.path.exists(base_dir):
    print(f"Error: Directory {base_dir} does not exist!")

# Save files
file_names = {
    'df_finance': df_finance,
    'direct_parity': df_direct_parity,
    'meta_parity': df_meta_parity,
    'roi': roi,
    # 'dau': dau,
    'deal_share': df_deal_share,
    'bookability': df_bookability,
    'weekly': df_weekly
}

# Save with verification
for name, df in file_names.items():
    file_path = f'{base_dir}{name}.csv'
    print(f"Saving: {file_path}")
    # print(df.head())  # Print first few rows to confirm data
    df.to_csv(file_path, index=False)

# Read files
for name in file_names.keys():
    file_path = f'{base_dir}{name}.csv'
    print(f"Reading: {file_path}")

    if os.path.exists(file_path):
        df_read = pd.read_csv(file_path)
        # print(df_read.head())  # Print first few rows to compare
    else:
        print(f"Error: {file_path} not found!")

Saving: ../../data_file/df_finance.csv
Saving: ../../data_file/direct_parity.csv
Saving: ../../data_file/meta_parity.csv
Saving: ../../data_file/roi.csv
Saving: ../../data_file/deal_share.csv
Saving: ../../data_file/bookability.csv
Saving: ../../data_file/weekly.csv
Reading: ../../data_file/df_finance.csv
Reading: ../../data_file/direct_parity.csv
Reading: ../../data_file/meta_parity.csv
Reading: ../../data_file/roi.csv
Reading: ../../data_file/deal_share.csv
Reading: ../../data_file/bookability.csv
Reading: ../../data_file/weekly.csv


In [6]:
df_weekly

,wk_starting,wk_ending,us_travel_type,company,brand,offer_type,carrier_detail,carrier,offer_method_code,search_channel,...,yoy_gross_contribution,net_contribution_ly,net_contribution_cy,yoy_net_contribution,gr_contr_fee_ly,gr_contr_fee_cy,yoy_gr_contr_fee,net_contr_fee_ly,net_contr_fee_cy,yoy_net_contr_fee
0,2025-07-13,2025-07-19,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2025-07-13,2025-07-19,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2025-07-13,2025-07-19,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2025-07-13,2025-07-19,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2025-07-13,2025-07-19,Domestic,Agoda,Agoda,Flights Only,Alaska Airlines (AS),Alaska Airlines (AS),Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54294,2025-07-20,2025-07-26,X,Priceline B2C,Priceline,Packages,WestJet (WS),Other,Retail (Disclosed),Shop PPC,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54295,2025-07-20,2025-07-26,X,Priceline B2C,Priceline,Packages,airBaltic (BT),Other,Retail (Disclosed),Direct,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54296,2025-07-20,2025-07-26,X,Priceline B2C,Priceline,Packages,avianca (AV),Other,Retail (Disclosed),SEM_Brand,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54297,2025-07-20,2025-07-26,X,Priceline B2C,Priceline,Packages,interCaribbean (JY),Other,Retail (Disclosed),Affiliate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df_finance

,Period,Period_End,Scenario,Gross_Units,Net_Units,gross_cont_revenue_fee,net_cont_revenue_fee
0,1.Reporting Week,2025-07-26,ACTUALS,2.086600e+05,1.973800e+05,2.073761e+06,1.923916e+06
1,1.Reporting Week,2025-07-26,Z-Plan,2.368318e+05,2.201232e+05,2.138903e+06,2.134410e+06
2,2.Previous Week,2025-07-19,ACTUALS,2.213730e+05,2.068800e+05,2.160130e+06,1.984004e+06
3,2.Previous Week,2025-07-19,Z-Plan,2.470594e+05,2.295429e+05,2.140904e+06,2.143024e+06
4,3.MTD,2025-07-26,ACTUALS,8.111570e+05,7.601990e+05,8.203794e+06,7.517639e+06
5,3.MTD,2025-07-26,Z-Plan,9.061873e+05,8.419367e+05,7.857377e+06,7.871048e+06
6,4.QTD,2025-07-26,ACTUALS,8.111570e+05,7.601990e+05,8.203794e+06,7.517639e+06
7,4.QTD,2025-07-26,Z-Plan,9.061873e+05,8.419367e+05,7.857377e+06,7.871048e+06
8,5.YTD,2025-07-26,ACTUALS,6.311634e+06,5.888774e+06,6.731770e+07,6.171835e+07
9,5.YTD,2025-07-26,Z-Plan,7.042207e+06,6.539092e+06,6.033845e+07,6.049182e+07


In [8]:
df_finance.rename(columns={'gross_cont_revenue_fee': 'gross_cont_revenue', 'net_cont_revenue_fee': 'net_cont_revenue'}, inplace=True)
df_finance
df_updated = df_finance[['Gross_Units', 'Net_Units', 'gross_cont_revenue', 'net_cont_revenue']].astype({
    'Gross_Units': 'int',
    'Net_Units': 'int',
    'gross_cont_revenue': 'int',
    'net_cont_revenue': 'int'
})

# If you need to keep the rest of the DataFrame unchanged, you can join the other columns back
df_finance = df_finance.drop(['Gross_Units', 'Net_Units', 'gross_cont_revenue', 'net_cont_revenue'], axis=1).join(df_updated)
df_finance


,Period,Period_End,Scenario,Gross_Units,Net_Units,gross_cont_revenue,net_cont_revenue
0,1.Reporting Week,2025-07-26,ACTUALS,208660,197380,2073761,1923916
1,1.Reporting Week,2025-07-26,Z-Plan,236831,220123,2138902,2134409
2,2.Previous Week,2025-07-19,ACTUALS,221373,206880,2160130,1984004
3,2.Previous Week,2025-07-19,Z-Plan,247059,229542,2140904,2143024
4,3.MTD,2025-07-26,ACTUALS,811157,760199,8203793,7517639
5,3.MTD,2025-07-26,Z-Plan,906187,841936,7857377,7871048
6,4.QTD,2025-07-26,ACTUALS,811157,760199,8203793,7517639
7,4.QTD,2025-07-26,Z-Plan,906187,841936,7857377,7871048
8,5.YTD,2025-07-26,ACTUALS,6311634,5888774,67317703,61718350
9,5.YTD,2025-07-26,Z-Plan,7042206,6539092,60338447,60491818


In [9]:

def format_number(num):
    if pd.isna(num):
        return ''
    if abs(num) >= 1e6:
        return f"{num/1e6:.1f}M"
    elif abs(num) >= 1e3:
        return f"{num/1e3:.0f}K"
    elif abs(num) < 1e3:
        return "<1K"
    return f"{num:.0f}"

def format_percentage(num):
    if pd.isna(num):
        return ''
    return f"{num:.1f}%"

def round_to_nearest_10(num):
  return round(num / 10) * 10

df_pricelince=df_weekly[(df_weekly['brand']== 'Priceline')]
df_pricelince_air=df_weekly[(df_weekly['brand']== 'Priceline')&(df_weekly['offer_type']== 'Flights Only')]
df_pricelince_b2c=df_weekly[(df_weekly['brand']== 'Priceline')&(df_weekly['company']== 'Priceline B2C')]
df_pricelince_b2c_standalone=df_weekly[(df_weekly['brand']== 'Priceline')&(df_weekly['company']== 'Priceline B2C')&(df_weekly['offer_type']== 'Flights Only')]


In [10]:

# Calculate the percentages
def calculate_percentage(current, previous):
    return (((current - previous) / previous) * 100).round(1)

# Reporting Week
reporting_week_actuals = df_finance[df_finance['Period'] == '1.Reporting Week'].iloc[0]
reporting_week_plan = df_finance[df_finance['Period'] == '6.Reporting Week LY'].iloc[0]

# Previous Week
previous_week_actuals = df_finance[df_finance['Period'] == '2.Previous Week'].iloc[0]
previous_week_plan = df_finance[df_finance['Period'] == '7.Previous Week LY'].iloc[0]



# Calculate metrics
df_finance_ly = {
    'Metric': ['net_tkts_cy', 'gr_tkts_cy', 'net_contribution_cy'
               , 'gross_contribution_cy'
               ],
    'Actual': [
        reporting_week_actuals['Net_Units'],
        reporting_week_actuals['Gross_Units'],
        reporting_week_actuals['net_cont_revenue'],
        reporting_week_actuals['gross_cont_revenue']
    ],
    'Reporting Week': [
        calculate_percentage(reporting_week_actuals['Net_Units'], reporting_week_plan['Net_Units']),
        calculate_percentage(reporting_week_actuals['Gross_Units'], reporting_week_plan['Gross_Units']),
        calculate_percentage(reporting_week_actuals['net_cont_revenue'], reporting_week_plan['net_cont_revenue']),
        # None
        calculate_percentage(reporting_week_actuals['gross_cont_revenue'], reporting_week_plan['gross_cont_revenue'])
    ],
    'Previous Week': [
        calculate_percentage(previous_week_actuals['Net_Units'], previous_week_plan['Net_Units']),
        calculate_percentage(previous_week_actuals['Gross_Units'], previous_week_plan['Gross_Units']),
        calculate_percentage(previous_week_actuals['net_cont_revenue'], previous_week_plan['net_cont_revenue']),
        # None
        calculate_percentage(previous_week_actuals['gross_cont_revenue'], previous_week_plan['gross_cont_revenue'])
    ]

}

df_finance_ly= pd.DataFrame(df_finance_ly)
df_finance_ly['Actual']=df_finance_ly['Actual'].apply(format_number)
df_finance_ly['Reporting Week']=df_finance_ly['Reporting Week'].apply(format_percentage)
df_finance_ly['Previous Week']=df_finance_ly['Previous Week'].apply(format_percentage)
df_finance_ly


,Metric,Actual,Reporting Week,Previous Week
0,net_tkts_cy,197K,-6.5%,-8.5%
1,gr_tkts_cy,209K,-8.3%,-8.9%
2,net_contribution_cy,1.9M,-17.9%,-17.7%
3,gross_contribution_cy,2.1M,-18.1%,-16.0%


In [11]:

# Calculate the percentages
def calculate_percentage(current, previous):
    return (((current - previous) / previous) * 100).round(1)

# Reporting Week
reporting_week_actuals = df_finance[df_finance['Period'] == '1.Reporting Week'].iloc[0]
reporting_week_plan = df_finance[df_finance['Period'] == '1.Reporting Week'].iloc[1]

# Previous Week
previous_week_actuals = df_finance[df_finance['Period'] == '2.Previous Week'].iloc[0]
previous_week_plan = df_finance[df_finance['Period'] == '2.Previous Week'].iloc[1]

# MTD
mtd_actuals = df_finance[df_finance['Period'] == '3.MTD'].iloc[0]
mtd_plan = df_finance[df_finance['Period'] == '3.MTD'].iloc[1]


qtd_actuals = df_finance[df_finance['Period'] == '4.QTD'].iloc[0]
qtd_plan = df_finance[df_finance['Period'] == '4.QTD'].iloc[1]

ytd_actuals = df_finance[df_finance['Period'] == '5.YTD'].iloc[0]
ytd_plan = df_finance[df_finance['Period'] == '5.YTD'].iloc[1]

# Calculate metrics
finance_plan_data = {
    'Metric': ['net_tkts_cy', 'gr_tkts_cy', 'net_contribution_cy'
            #    , 'gross_contribution_cy'
               ],
    'Reporting Week': [
        calculate_percentage(reporting_week_actuals['Net_Units'], reporting_week_plan['Net_Units']),
        calculate_percentage(reporting_week_actuals['Gross_Units'], reporting_week_plan['Gross_Units']),
        calculate_percentage(reporting_week_actuals['net_cont_revenue'], reporting_week_plan['net_cont_revenue'])
        #,calculate_percentage(reporting_week_actuals['gross_cont_revenue'], reporting_week_plan['gross_cont_revenue'])
    ],
    'Previous Week': [
        calculate_percentage(previous_week_actuals['Net_Units'], previous_week_plan['Net_Units']),
        calculate_percentage(previous_week_actuals['Gross_Units'], previous_week_plan['Gross_Units']),
        calculate_percentage(previous_week_actuals['net_cont_revenue'], previous_week_plan['net_cont_revenue'])
        # ,calculate_percentage(previous_week_actuals['gross_cont_revenue'], previous_week_plan['gross_cont_revenue'])
    ],
    'MTD': [
        calculate_percentage(mtd_actuals['Net_Units'], mtd_plan['Net_Units']),
        calculate_percentage(mtd_actuals['Gross_Units'], mtd_plan['Gross_Units']),
        calculate_percentage(mtd_actuals['net_cont_revenue'], mtd_plan['net_cont_revenue'])
        #,calculate_percentage(mtd_actuals['gross_cont_revenue'], mtd_plan['gross_cont_revenue'])
    ],
    'QTD': [
        calculate_percentage(qtd_actuals['Net_Units'], qtd_plan['Net_Units']),
        calculate_percentage(qtd_actuals['Gross_Units'], qtd_plan['Gross_Units']),
        calculate_percentage(qtd_actuals['net_cont_revenue'], qtd_plan['net_cont_revenue'])
        #,calculate_percentage(qtd_actuals['gross_cont_revenue'], qtd_plan['gross_cont_revenue'])
    ],
    'YTD': [
        calculate_percentage(ytd_actuals['Net_Units'], ytd_plan['Net_Units']),
        calculate_percentage(ytd_actuals['Gross_Units'], ytd_plan['Gross_Units']),
        calculate_percentage(ytd_actuals['net_cont_revenue'], ytd_plan['net_cont_revenue'])
        # , calculate_percentage(ytd_actuals['gross_cont_revenue'], ytd_plan['gross_cont_revenue'])
    ]
}

df_finance_plan = pd.DataFrame(finance_plan_data)
df_finance_plan['Reporting Week']=df_finance_plan['Reporting Week'].apply(format_percentage)
df_finance_plan['Previous Week']=df_finance_plan['Previous Week'].apply(format_percentage)
df_finance_plan['MTD']=df_finance_plan['MTD'].apply(format_percentage)
df_finance_plan['QTD']=df_finance_plan['QTD'].apply(format_percentage)
df_finance_plan['YTD']=df_finance_plan['YTD'].apply(format_percentage)
df_finance_plan


,Metric,Reporting Week,Previous Week,MTD,QTD,YTD
0,net_tkts_cy,-10.3%,-9.9%,-9.7%,-9.7%,-9.9%
1,gr_tkts_cy,-11.9%,-10.4%,-10.5%,-10.5%,-10.4%
2,net_contribution_cy,-9.9%,-7.4%,-4.5%,-4.5%,2.0%


In [12]:
df_finance_result= pd.merge(df_finance_ly, df_finance_plan,how='left', on='Metric',suffixes=('','pl'))
df_finance_result = df_finance_result.rename(columns={
        'Reporting Week_ly': 'Reporting Week',
        'Previous Week_ly': 'Previous Week'
        })
df_finance_result=df_finance_result.fillna('')
df_finance_result

,Metric,Actual,Reporting Week,Previous Week,Reporting Weekpl,Previous Weekpl,MTD,QTD,YTD
0,net_tkts_cy,197K,-6.5%,-8.5%,-10.3%,-9.9%,-9.7%,-9.7%,-9.9%
1,gr_tkts_cy,209K,-8.3%,-8.9%,-11.9%,-10.4%,-10.5%,-10.5%,-10.4%
2,net_contribution_cy,1.9M,-17.9%,-17.7%,-9.9%,-7.4%,-4.5%,-4.5%,2.0%
3,gross_contribution_cy,2.1M,-18.1%,-16.0%,,,,,


# TSA

In [13]:
from tsa_table import create_tsa_web_table
df_tsa_web=create_tsa_web_table()
df_tsa_web=df_tsa_web.dropna()

In [14]:
import tsa_table
import importlib

importlib.reload(tsa_table)
from tsa_table import create_df_tsa_table

df_tsa=create_df_tsa_table(tsa_data,df_tsa_web,format_percentage,end_date,pp_end_date,cwly_date,pwly_date)
df_tsa


,priceline_passengers,TSA_passengers,%,YoY
CY,272148.0,19615425.0,1.4%,-5.74
LY,281971.0,19480900.0,1.4%,NaN
CY_PW,270149.0,19602214.0,1.4%,-6.37
LY_PW,278013.0,19256531.0,1.4%,NaN


In [15]:

import subsummary_table as subsummary_table
import importlib

importlib.reload(subsummary_table)
from subsummary_table import create_subsummary_table

df_subsummary = create_subsummary_table(df_pricelince,df_pricelince_air,format_number)
df_normalized=df_subsummary.tail(2)
# .reset_index(names=['Metric'])
df_normalized



,Actual,Reporting Week,Previous Week
normalized_net_tickets_cy,167K,-10.3%,-10.7%
normalized_gross_tickets_cy,176K,-12.0%,-11.0%


In [16]:
df_normalized.loc['DAU','Actual']=0
df_normalized.loc['DAU','Actual']=0
df_normalized.loc['DAU','Reporting Week']=0
df_normalized.loc['DAU','Previous Week']=0

df_normalized.loc['TSA','Actual']=df_tsa.loc['CY','%']
df_normalized.loc['TSA','Actual']=df_tsa.loc['CY','%']
df_normalized.loc['TSA','Reporting Week']=format_percentage(df_tsa.loc['CY','YoY'])
df_normalized.loc['TSA','Previous Week']=format_percentage(df_tsa.loc['CY_PW','YoY'])

df_normalized=df_normalized.reset_index(names=['Metric'])
df_normalized

/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_88608/854525455.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_normalized.loc['DAU','Actual']=0


,Metric,Actual,Reporting Week,Previous Week
0,normalized_net_tickets_cy,167K,-10.3%,-10.7%
1,normalized_gross_tickets_cy,176K,-12.0%,-11.0%
2,DAU,0,0,0
3,TSA,1.4%,-5.7%,-6.4%


In [17]:
df_summary=pd.concat([df_finance_result, df_normalized]).fillna('').set_index('Metric')
df_summary


,Actual,Reporting Week,Previous Week,Reporting Weekpl,Previous Weekpl,MTD,QTD,YTD
Metric,,,,,,,,
net_tkts_cy,197K,-6.5%,-8.5%,-10.3%,-9.9%,-9.7%,-9.7%,-9.9%
gr_tkts_cy,209K,-8.3%,-8.9%,-11.9%,-10.4%,-10.5%,-10.5%,-10.4%
net_contribution_cy,1.9M,-17.9%,-17.7%,-9.9%,-7.4%,-4.5%,-4.5%,2.0%
gross_contribution_cy,2.1M,-18.1%,-16.0%,,,,,
normalized_net_tickets_cy,167K,-10.3%,-10.7%,,,,,
normalized_gross_tickets_cy,176K,-12.0%,-11.0%,,,,,
DAU,0,0,0,,,,,
TSA,1.4%,-5.7%,-6.4%,,,,,


In [18]:
import kpi
import importlib

importlib.reload(kpi)

from kpi import calculate_business_metrics

# Calculate business metrics
df_business = calculate_business_metrics(df_pricelince,end_date, pp_end_date)

from kpi import calculate_carrier_metrics
# Calculate carrier metrics
df_carrier = calculate_carrier_metrics(df_pricelince_b2c_standalone, end_date, pp_end_date)

from kpi import calculate_channel_metrics
# Calculate channel metrics
df_channel = calculate_channel_metrics(df_pricelince_b2c_standalone, end_date, pp_end_date)

from kpi import calculate_source_metrics
# Calculate source metrics
df_source = calculate_source_metrics(df_pricelince_b2c_standalone, end_date, pp_end_date)

from kpi import calculate_brand_metrics
# Calculate brand metrics
df_brand = calculate_brand_metrics(df_weekly, end_date, pp_end_date)


In [19]:
df_channel


,Net Tickets_App,YoY_App,YoY PW_App,Net Tickets_Desk/MWEB,YoY_Desk/MWEB,YoY PW_Desk/MWEB,Net Tickets_Total,YoY_Total,YoY PW_Total
search_channel_group,,,,,,,,,
Direct,34K,-6.8%,-7.8%,25K,-17.7%,-20.8%,59K,-11.8%,-13.9%
Web Marketing,7K,34.2%,34.1%,54K,-6.6%,-13.0%,60K,-3.4%,-9.7%
Shop PPC,7K,-7.4%,-14.7%,19K,-20.0%,-16.7%,26K,-16.9%,-16.2%
Affiliate,<1K,77.5%,96.0%,11K,-22.2%,-18.3%,11K,-21.5%,-17.6%
Total,48K,-2.5%,-4.7%,109K,-13.6%,-16.1%,157K,-10.5%,-13.0%


In [20]:
df_carrier

,Net Tickets_Retail,YoY_Retail,YoY PW_Retail,Net Tickets_Opaque,YoY_Opaque,YoY PW_Opaque,Net Tickets_Total,YoY_Total,YoY PW_Total
carrier,,,,,,,,,
American Airlines (AA),32K,-12.9%,-17.7%,12K,2.3%,-15.1%,44K,-9.4%,-17.1%
Delta Air Lines (DL),23K,-7.1%,-13.2%,<1K,-87.0%,-82.9%,24K,-21.6%,-23.8%
United Airlines (UA),16K,-5.3%,-6.7%,7K,-11.2%,3.6%,22K,-7.1%,-3.9%
Spirit Airlines (NK),14K,-30.9%,-24.9%,,,,14K,-30.9%,-24.9%
Frontier Airlines (F9),18K,9.5%,-5.6%,,,,18K,9.5%,-5.6%
Alaska Airlines (AS),4K,-17.1%,-21.6%,3K,35.0%,20.2%,8K,0.2%,-6.4%
JetBlue Airways (B6),6K,-7.6%,11.5%,<1K,inf%,3300.0%,6K,-7.4%,11.9%
Other,21K,-3.1%,-3.6%,<1K,783.3%,474.4%,21K,-1.7%,-2.6%
Total,134K,-9.6%,-12.2%,23K,-15.5%,-17.5%,157K,-10.5%,-13.0%


In [21]:
df_channel

,Net Tickets_App,YoY_App,YoY PW_App,Net Tickets_Desk/MWEB,YoY_Desk/MWEB,YoY PW_Desk/MWEB,Net Tickets_Total,YoY_Total,YoY PW_Total
search_channel_group,,,,,,,,,
Direct,34K,-6.8%,-7.8%,25K,-17.7%,-20.8%,59K,-11.8%,-13.9%
Web Marketing,7K,34.2%,34.1%,54K,-6.6%,-13.0%,60K,-3.4%,-9.7%
Shop PPC,7K,-7.4%,-14.7%,19K,-20.0%,-16.7%,26K,-16.9%,-16.2%
Affiliate,<1K,77.5%,96.0%,11K,-22.2%,-18.3%,11K,-21.5%,-17.6%
Total,48K,-2.5%,-4.7%,109K,-13.6%,-16.1%,157K,-10.5%,-13.0%


In [22]:
df_source

,Net Tickets_Published,YoY_Published,YoY PW_Published,Net Tickets_Private,YoY_Private,YoY PW_Private,Net Tickets_Total,YoY_Total,YoY PW_Total
gds_booking_category,,,,,,,,,
Direct Connect,77K,5.1%,-3.9%,11K,165.7%,110.2%,88K,13.9%,3.6%
Indirect Connect,52K,-27.2%,-25.3%,15K,-38.7%,-32.1%,67K,-30.1%,-26.9%
Phone Sales,,,,1K,-17.9%,-5.0%,1K,-17.9%,-5.0%
Total,129K,-10.9%,-14.4%,27K,-8.5%,-6.0%,157K,-10.5%,-13.0%


# MOR


In [23]:
# Create df_mor
df_mor= pd.DataFrame()
df_mor['Net Tickets'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor['Net Tickets_cwly'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)
df_mor['Net Tickets_PW'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor['Net Tickets_PWly'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)& ((df_pricelince['merchant_of_record'] == 'VCC')|(df_pricelince['merchant_of_record'] == 'PCLN'))].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)


# Calculate YoY and YoY PW
df_mor.loc['Total','Net Tickets':'Net Tickets_PWly']=[df_mor['Net Tickets'].sum(),df_mor['Net Tickets_cwly'].sum(),df_mor['Net Tickets_PW'].sum(), df_mor['Net Tickets_PWly'].sum()]


# Create df_mor_1
df_mor_1= pd.DataFrame()
df_mor_1['Net Tickets'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor_1['Net Tickets_cwly'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)
df_mor_1['Net Tickets_PW'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['offer_method_code'])['net_tkts_cy'].sum().round(-1).astype(int)
df_mor_1['Net Tickets_PWly'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['offer_method_code'])['net_tkts_ly'].sum().round(-1).astype(int)
# Calculate YoY and YoY PW
df_mor_1.loc['Total','Net Tickets':'Net Tickets_PWly']=[df_mor_1['Net Tickets'].sum(),df_mor_1['Net Tickets_cwly'].sum(),df_mor_1['Net Tickets_PW'].sum(), df_mor_1['Net Tickets_PWly'].sum()]

df_mor = pd.concat([df_mor,df_mor_1.add_suffix('_total')], axis=1)


df_mor['Actual']=(df_mor['Net Tickets']/df_mor['Net Tickets_total']*100).round(1).apply(format_percentage)
df_mor['%mor_ly']=(df_mor['Net Tickets_cwly']/df_mor['Net Tickets_cwly_total']*100).round().apply(format_percentage)
df_mor['YoY_bps']=(((df_mor['Net Tickets']/df_mor['Net Tickets_total']*100)-(df_mor['Net Tickets_cwly']/df_mor['Net Tickets_cwly_total']*100))*100).round()
df_mor['Actual_PW']=(df_mor['Net Tickets_PW']/df_mor['Net Tickets_PW_total']*100).round().apply(format_percentage)
df_mor['YoY_PW_bps']=(((df_mor['Net Tickets_PW']/df_mor['Net Tickets_PW_total']*100)-(df_mor['Net Tickets_PWly']/df_mor['Net Tickets_PWly_total']*100))*100).round()
df_mor['YoY_bps']=round_to_nearest_10(df_mor['YoY_bps'])
df_mor['YoY_PW_bps']=round_to_nearest_10(df_mor['YoY_PW_bps'])
df_mor=df_mor.loc[['Retail (Disclosed)', 'Total'],['Actual','YoY_bps','YoY_PW_bps']]
df_mor


,Actual,YoY_bps,YoY_PW_bps
offer_method_code,,,
Retail (Disclosed),16.5%,1030.0,1100.0
Total,26.1%,800.0,860.0


In [24]:
# Create df_mor
df_mor_2= pd.DataFrame()
df_mor_2['Net Tickets'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['us_travel_type'])['net_tkts_cy'].sum().astype(int)
df_mor_2['Net Tickets_cwly'] = df_pricelince[(df_pricelince['wk_ending'] == end_date)].groupby(['us_travel_type'])['net_tkts_ly'].sum().astype(int)
df_mor_2['Net Tickets_PW'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['us_travel_type'])['net_tkts_cy'].sum().astype(int)
df_mor_2['Net Tickets_PWly'] = df_pricelince[(df_pricelince['wk_ending'] == pp_end_date)].groupby(['us_travel_type'])['net_tkts_ly'].sum().astype(int)
df_mor_2.loc['Total','Net Tickets':'Net Tickets_PWly']=[df_mor_2['Net Tickets'].sum(),df_mor_2['Net Tickets_cwly'].sum(),df_mor_2['Net Tickets_PW'].sum(), df_mor_2['Net Tickets_PWly'].sum()]
actual_cy=(df_mor_2.loc['US Outbound','Net Tickets']*100/df_mor_2.loc['Total','Net Tickets']).round()
actual_ly=df_mor_2.loc['US Outbound','Net Tickets_cwly']*100/df_mor_2.loc['Total','Net Tickets_cwly']
actual_PW=df_mor_2.loc['US Outbound','Net Tickets_PW']*100/df_mor_2.loc['Total','Net Tickets_PW']
actual_PWly=df_mor_2.loc['US Outbound','Net Tickets_PWly']*100/df_mor_2.loc['Total','Net Tickets_PWly']
df_mor.loc['US Outbound','Actual']=format_percentage(actual_cy)
df_mor.loc['US Outbound','YoY_bps']=round_to_nearest_10((actual_cy-actual_ly)*100)
df_mor.loc['US Outbound','YoY_PW_bps']=round_to_nearest_10((actual_PW-actual_PWly)*100)
new_index_order = ['US Outbound','Retail (Disclosed)','Total']
df_mor=df_mor.reindex(new_index_order).fillna('')
df_mor

,Actual,YoY_bps,YoY_PW_bps
offer_method_code,,,
US Outbound,9.0%,30.0,40.0
Retail (Disclosed),16.5%,1030.0,1100.0
Total,26.1%,800.0,860.0


## ROI

In [25]:

import roi_conversion_table

import importlib
importlib.reload(roi_conversion_table)


from roi_conversion_table import create_roi_table

df_roi = create_roi_table(roi,end_date,format_number)

df_roi



,Cost_App,Cost YoY_App,ROI_App,Cost_Desk/MWEB,Cost YoY_Desk/MWEB,ROI_Desk/MWEB,Cost_Total,Cost YoY_Total,ROI_Total
channel,,,,,,,,,
Direct,,,,,,,,,
SEM Core,,,,757K,-5.7%,0.99,757K,-5.7%,0.99
SEM Brand,,,,81K,24.4%,5.2,81K,24.4%,5.2
Shop PPC,,,,,,,,,
Affiliate,,,,,,,,,
Email,,,,,,,,,
Meta,,,,17K,-28.1%,1.33,17K,-28.1%,1.33
Total,<1K,,,856K,-4.1%,1.39,856K,-4.1%,1.39


In [26]:
# from roi_conversion_table import create_conversion_table

# df_conversion = create_conversion_table(dau,end_date,format_number)
# df_conversion



## Parity data

In [27]:
from parity_table import create_parity_table

df_parity = create_parity_table(
        format_percentage,
        df_direct_parity,
        round_to_nearest_10,
        df_meta_parity)
df_parity

,Actual_pcln,YoY (bps)_pcln,YoY PW (bps)_pcln,Actual_expe,YoY (bps)_expe,YoY PW (bps)_expe
Direct vs Expedia,94.7%,510.0,290.0,NaN,NaN,NaN
Kayak Placement,16.3%,300.0,100.0,25.5%,1460.0,1240.0
Skyscanner Placement,20.6%,1460.0,1410.0,29.6%,-1660.0,-1580.0


In [28]:
df_deal_share

,deal_vs_non_deal,yr_mnth,wk_ending,offer_type,fare_type,totalTkts
0,Deal,2023-01,2023-01-07,Air,Web Fare,2790.0
1,Deal,2023-01,2023-01-07,Air,Retail Net,127.0
2,Deal,2023-01,2023-01-07,Air,Opaque,13650.0
3,Deal,2023-01,2023-01-07,Air,Package Net,1025.0
4,No Deal,2023-01,2023-01-07,Air,Retail Published,95674.0
...,...,...,...,...,...,...
2199,No Deal,2025-07,2025-07-26,PKG,Retail Published,16572.0
2200,Deal,2025-07,2025-07-26,PKG,Retail Net,39.0
2201,Deal,2025-07,2025-07-26,PKG,Web Fare,1742.0
2202,Deal,2025-07,2025-07-26,PKG,Mixed,7.0


In [29]:
deal_ticket_actual_cy=df_deal_share[(df_deal_share['wk_ending']==end_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_cy=deal_ticket_actual_cy[0]/(deal_ticket_actual_cy[0]+deal_ticket_actual_cy[1])
# deal_ticket_actual_cy=format_percentage(deal_ticket_actual_cy*100)

deal_ticket_actual_cy

deal_ticket_actual_pw=df_deal_share[(df_deal_share['wk_ending']==pp_end_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_pw=deal_ticket_actual_pw[0]/(deal_ticket_actual_pw[0]+deal_ticket_actual_pw[1])
# deal_ticket_actual_pw=format_percentage(deal_ticket_actual_pw*100)
deal_ticket_actual_pw

deal_ticket_actual_cwly=df_deal_share[(df_deal_share['wk_ending']==cwly_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_cwly=deal_ticket_actual_cwly[0]/(deal_ticket_actual_cwly[0]+deal_ticket_actual_cwly[1])
# deal_ticket_actual_cwly=format_percentage(deal_ticket_actual_cwly*100)
deal_ticket_actual_cwly

deal_ticket_actual_pwly=df_deal_share[(df_deal_share['wk_ending']==pwly_date)].groupby('deal_vs_non_deal')['totalTkts'].sum()
deal_ticket_actual_pwly=deal_ticket_actual_pwly[0]/(deal_ticket_actual_pwly[0]+deal_ticket_actual_pwly[1])
# deal_ticket_actual_pwly=format_percentage(deal_ticket_actual_pwly*100)
deal_ticket_actual_pwly

/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_88608/2117931116.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  deal_ticket_actual_cy=deal_ticket_actual_cy[0]/(deal_ticket_actual_cy[0]+deal_ticket_actual_cy[1])
/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_88608/2117931116.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  deal_ticket_actual_pw=deal_ticket_actual_pw[0]/(deal_ticket_actual_pw[0]+deal_ticket_actual_pw[1])
/var/folders/k6/ygnww5yd0rx_9c55x7xb7xs00000gn/T/ipykernel_88608/2117931116.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future

np.float64(0.33003086237738594)

## MIDT

In [30]:



# df_midt_data= pd.DataFrame()
# #Calulate actual
# a=df_midt[(df_midt['wk_ending']==end_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==end_date)]['Tickets'].sum()
# # Calculate the percentage
# Actual = ((a * 100) / b)


# #Calulate cwly_date
# a=df_midt[(df_midt['wk_ending']==cwly_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==cwly_date)]['Tickets'].sum()
# # Calculate the percentage
# Actual_cwly = ((a * 100) / b)

# #Calulate PWly_date
# a=df_midt[(df_midt['wk_ending']==pp_end_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==pp_end_date)]['Tickets'].sum()
# # Calculate the percentage
# actual_PW = ((a * 100) / b)

# #Calulate PWly_date
# a=df_midt[(df_midt['wk_ending']==PWly_date)].groupby(['agency'])['Tickets'].sum()['PRICELINE']
# b=df_midt[(df_midt['wk_ending']==PWly_date)]['Tickets'].sum()
# # Calculate the percentage
# actual_PWly = ((a * 100) / b)


# # Create a new DataFrame with the calculated 'Actual' column
# df_midt_data = pd.DataFrame()
# df_midt_data['Actual']=[format_percentage(Actual)]
# df_midt_data['YoY (bps)']=[round_to_nearest_10((Actual-Actual_cwly)*100)]
# df_midt_data['YoY PW(bps)']=[round_to_nearest_10((actual_PW-actual_PWly)*100)]
# df_midt_data
# # df_midt_data['Actual']

## SEM

In [31]:
# df_sem=df_sem[(df_sem['DOMAIN']=='You')|(df_sem['DOMAIN']=='expedia.com')]
# df_sem
# # Calculate actuals for current, cwly, pp, and pwly dates
# def compute_rate_ratio(df, date):
#     grouped = df[df['wk_ending'] == date].groupby('DOMAIN')[['ABS_TOP_PAGE_RATE_IMPRESSIONS', 'TOTAL_IMPRESSIONS']].sum()
#     grouped['RATE_RATIO'] = grouped['ABS_TOP_PAGE_RATE_IMPRESSIONS'] / grouped['TOTAL_IMPRESSIONS']
#     return grouped['RATE_RATIO']  # return only the RATE_RATIO series

# # Compute each set
# actual = compute_rate_ratio(df_sem, end_date)
# actual_cwly = compute_rate_ratio(df_sem, cwly_date)
# actual_pw = compute_rate_ratio(df_sem, pp_end_date)
# actual_pwly = compute_rate_ratio(df_sem, pwly_date)

# df_sem_data = pd.DataFrame({
#     'Actual': (actual*100).apply(format_percentage),
#     'YoY (bps)': round_to_nearest_10((actual - actual_cwly) * 10000),
#     'YoY PW(bps)': round_to_nearest_10((actual_pw - actual_pwly) * 10000)
# })

# df_sem_data.reset_index(inplace=True)  # optional, for clarity if you need DOMAIN as a column
# df_sem_data


In [32]:
df_others = pd.DataFrame()
df_others.loc['deal_share','Actual_pcln']=format_percentage(deal_ticket_actual_cy*100)
df_others.loc['deal_share','YoY (bps)_pcln']=round_to_nearest_10((deal_ticket_actual_cy-deal_ticket_actual_cwly)*10000)
df_others.loc['deal_share','YoY PW (bps)_pcln']=round_to_nearest_10((deal_ticket_actual_pw-deal_ticket_actual_pwly)*10000)


# df_others.loc['MIDT','Actual_pcln']=df_midt_data.loc[0,'Actual']
# df_others.loc['MIDT','YoY (bps)_pcln']=df_midt_data.loc[0,'YoY (bps)']
# df_others.loc['MIDT','YoY PW (bps)_pcln']=df_midt_data.loc[0,'YoY PW(bps)']

# df_others.loc['SEM Impressions','Actual_pcln']=df_sem_data.loc[0,'Actual']
# df_others.loc['SEM Impressions','YoY (bps)_pcln']=df_sem_data.loc[0,'YoY (bps)']
# df_others.loc['SEM Impressions','YoY PW (bps)_pcln']=df_sem_data.loc[0,'YoY PW(bps)']

# df_others.loc['SEM Impressions','Actual_expe']=df_sem_data.loc[1,'Actual']
# df_others.loc['SEM Impressions','YoY (bps)_expe']=df_sem_data.loc[1,'YoY (bps)']
# df_others.loc['SEM Impressions','YoY PW (bps)_expe']=df_sem_data.loc[1,'YoY PW(bps)']

df_others.loc['TSA','Actual_pcln']=df_tsa.loc['CY','%']
df_others.loc['TSA','YoY (bps)_pcln']=df_tsa.loc['CY','YoY']
df_others.loc['TSA','YoY PW (bps)_pcln']=df_tsa.loc['CY_PW','YoY']

df_others.loc['US Outbound','Actual_pcln']=df_mor.loc['US Outbound','Actual']
df_others.loc['US Outbound','YoY (bps)_pcln']=df_mor.loc['US Outbound','YoY_bps']
df_others.loc['US Outbound','YoY PW (bps)_pcln']=df_mor.loc['US Outbound','YoY_PW_bps']
df_others.loc['Merchant Retail','Actual_pcln']=df_mor.loc['Retail (Disclosed)','Actual']
df_others.loc['Merchant Retail','YoY (bps)_pcln']=df_mor.loc['Retail (Disclosed)','YoY_bps']
df_others.loc['Merchant Retail','YoY PW (bps)_pcln']=df_mor.loc['Retail (Disclosed)','YoY_PW_bps']
df_others.loc['Merchant Total','Actual_pcln']=df_mor.loc['Total','Actual']
df_others.loc['Merchant Total','YoY (bps)_pcln']=df_mor.loc['Total','YoY_bps']
df_others.loc['Merchant Total','YoY PW (bps)_pcln']=df_mor.loc['Total','YoY_PW_bps']


# df_others = df_others.append(df_parity.iloc[0:])
df_others= pd.concat([df_others, df_parity.iloc[0:]])
new_index_order = [ 'ARC','MIDT','SEM Impressions','Direct vs Expedia','Kayak Placement','Skyscanner Placement', 'US Outbound','Merchant Retail','Merchant Total','deal_share','Bookability']
df_others=df_others.reindex(new_index_order).fillna('')
df_others

,Actual_pcln,YoY (bps)_pcln,YoY PW (bps)_pcln,Actual_expe,YoY (bps)_expe,YoY PW (bps)_expe
ARC,,,,,,
MIDT,,,,,,
SEM Impressions,,,,,,
Direct vs Expedia,94.7%,510.0,290.0,,,
Kayak Placement,16.3%,300.0,100.0,25.5%,1460.0,1240.0
Skyscanner Placement,20.6%,1460.0,1410.0,29.6%,-1660.0,-1580.0
US Outbound,9.0%,30.0,40.0,,,
Merchant Retail,16.5%,1030.0,1100.0,,,
Merchant Total,26.1%,800.0,860.0,,,
deal_share,34.6%,130.0,70.0,,,


In [33]:

import config_table
import importlib

importlib.reload(config_table)
from docx import Document
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx.shared import Pt
from config_table import clear_document
from config_table import set_font
from config_table import create_word_table
from config_table import create_summary_table
from config_table import create_others_table
from config_table import create_dau_table

# Data preparation and configurations
carrier_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/Retail.png', 'title': 'Retail'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/Express.png', 'title': 'Express Deals'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

business_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/Standalone.png', 'title': 'Standalone'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/Package.png', 'title': 'Package'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

channel_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/App.png', 'title': 'App'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/MWeb Desktop.png', 'title': 'MWeb/Desktop'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

source_logos = [
    {'start_col': 1, 'end_col': 3, 'path': '../Screenshots/weeklynote/Published.png', 'title': 'Published'},
    {'start_col': 4, 'end_col': 6, 'path': '../Screenshots/weeklynote/Private.png', 'title': 'Private'},
    {'start_col': 7, 'end_col': 9, 'path': '../Screenshots/weeklynote/Total.png', 'title': 'Total'}
]

def customize_bullet_style(paragraph, font_size):
    """
    Customize bullet style in a paragraph using XML manipulation.
    """
    # Access the paragraph's properties
    pPr = paragraph._element.get_or_add_pPr()
    numPr = OxmlElement('w:numPr')
    ilvl = OxmlElement('w:ilvl')
    ilvl.set(qn('w:val'), '0')  # Set indentation level
    numId = OxmlElement('w:numId')
    numId.set(qn('w:val'), '1')  # Set numbering ID

    numPr.append(ilvl)
    numPr.append(numId)
    pPr.append(numPr)

    # Modify font properties
    for run in paragraph.runs:
        run.font.name = "Montserrat"
        run.font.size = font_size

# Create a Word Document
word_document = Document()

# Clear the document (if necessary)
clear_document(word_document)

# Add Title
word_document.add_paragraph()
word_document.add_paragraph(f'Summary')
set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
word_document.add_paragraph('\n')

# Create Summary Table
create_summary_table(word_document, df_summary)

# Add General Notes
summary_notes = [
    'All Priceline tickets (includes B2B, Package, Express Deals, Phone Sales)',
    'Normalized tickets are counted the same as tickets, except split tickets count as 1 instead of 2',
    'Refunds are assigned to refund date',
    'Revenue is contribution w/fee + GDS incentives.  Does not include package or phone sales.',
    'Daily Active Users: engaged customers in GA4',
    'TSA market share: travel date; numerator counts all slices (OW is 1 slice; RT is 2 slices) where the first segment is either US domestic or US outbound (includes Agoda, Booking, Rocket Travel, White Labels, Phone Sales); denominator includes all people passing through TSA screening machines'
]
for note in summary_notes:
    word_document.add_paragraph(note, style='List Bullet')
    set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
    customize_bullet_style(word_document.paragraphs[-1], font_size=Pt(7))

# Add Tables for Business, Carrier, Channel, and Source
tables = [
    (df_business, "Business", business_logos, [
        'All Priceline tickets (includes Express Deals and Phone Sales, not normalized)'
    ]),
    (df_carrier, "Carrier", carrier_logos, [
        'Priceline (including phone sales), B2C, Standalone (not normalized)'
    ]),
    (df_channel, "Channel", channel_logos, [
        'Priceline (including phone sales), B2C, Standalone (includes Express Deals, not normalized)'
    ]),
    (df_source, "Source", source_logos, [
        'Priceline (including phone sales), B2C, Standalone (includes Express Deals, not normalized)',
        'Indirect Connect = GDS + Aggregators + Consolidators + NDC-X'
    ])
]
for df, title, logos, notes in tables:
    if df is None:
        continue

   # Add specific custom notes for each table
    if title == "Business":
        custom_note = "Total Business - Detail"
    elif title == "Carrier":
        custom_note = "Priceline B2C Standalone - Detail"
    else:
        custom_note = " "

    # Add the custom note before the table
    word_document.add_paragraph(custom_note)
    set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
    word_document.add_paragraph()

    create_word_table(df, title, logos, word_document)

    # word_document.add_paragraph()

    for note in notes:
        word_document.add_paragraph(note, style='List Bullet')
        # word_document.add_paragraph('\n')
        set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)

word_document.add_paragraph('\n')
word_document.add_paragraph('Other Metrics')
set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
create_others_table(word_document, df_others)

others_notes = [
    'ARC market share: OTA only, US POS, does not include NK/F9/SY, includes Agoda/Booking',
    'MIDT market share: GDS only, all carriers, US POS',
    'SEM impression share: percentage of eligible impressions we show at the absolute top of the ads',
    'Direct parity vs Expedia: US origins, win+tie rate',
    'Kayak placement: US origins, availability in the top 4 positions',
    'Skyscanner placement: US origins, availability in the top 4 positions',
    'US outbound share: ticket share for US origins, international destinations',
    'Merchant share: ticket share where Priceline is merchant of record',
    'Deal share: share of tickets sold as a deal',
    'Conversion: GA4, converted customers / engaged customers',
    'Bookability: acceptance rate, includes multiple attempts by same user'

]
for note in others_notes:
    word_document.add_paragraph(note, style='List Bullet')
    set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
word_document.add_paragraph('\n')

word_document.add_paragraph(f'ROI')
set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)
# Add ROI Section
roi_notes = [
    'ROI = Contribution / Cost',
    'SEM Brand includes all products, weighted by ratio of flight orders',
    'Meta includes Kayak/Momondo, includes Kayak credit',
    'Last week'
]

roi_table = [
    (df_roi, "Channel", channel_logos, roi_notes)
]

for df, title, logos, notes in roi_table:
    word_document.add_paragraph()
    word_document.add_paragraph('\n')
    create_word_table(df, title, logos, word_document)
    for note in notes:
        word_document.add_paragraph(note, style='List Bullet')
        set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
word_document.add_paragraph('\n')


# word_document.add_paragraph(f'DAU')
# set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(18), bold=True)

# # Add DAU Section
# dau_notes = [
#     'Search Stream',
#     'DAU = Daily Active Users, distinct customers searching on Priceline',
#     'Conversion Rate = Conversions / DAU',
#     'Last week'
# ]
# create_dau_table(word_document, df_conversion)

# for note in notes:
#         word_document.add_paragraph(note, style='List Bullet')
#         set_font(word_document.paragraphs[-1], font_name="Montserrat", font_size=Pt(7), bold=False)
# word_document.add_paragraph('\n')


# Save the Document
output_filename = os.path.join('../output/', f'Flight Performance Week Ending {end_date}.docx')
word_document.save(output_filename)
print(f"Word document '{output_filename}' has been created successfully.")

# Save PDF to Shared Drive
share_drive_path = '../../../Flight Weekly Note Output/'
word_document.save(share_drive_path + f'Flight Performance Week Ending {end_date}.pdf')

print(f"Word document saved to shared drive at '{share_drive_path}' has been created successfully.")


Word document '../output/Flight Performance Week Ending 2025-07-26.docx' has been created successfully.
Word document saved to shared drive at '../../../Flight Weekly Note Output/' has been created successfully.
